This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [100]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [101]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [102]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-02&end_date=2017-01-03&api_key=-mMbYGBrRuz6R6yJXsYU'
r = requests.get(url)
json_data = r.json()

In [103]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json_data)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-02', 'end_date': '2017-01-03', 'frequency': 'daily', 'data': [['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [104]:
url_full = f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}'
req = requests.get(url_full)
json_data_full = req.json()
json_data_full.keys()
AFX_dict = {}
AFX_dict = json_data_full.get("dataset_data")
AFX_dict.keys()


dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [105]:
col_names = AFX_dict.get("column_names")
col_names

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [106]:
data = AFX_dict.get('data')


In [107]:
AFX_dict_clean = {x:list(y) for x,y in zip(col_names, zip(*data))}

In [108]:
#Impute None values in open price with -1
for count, value in enumerate(AFX_dict_clean['Open']):
    if value == None:
        AFX_dict_clean['Open'][count] = -1
#AFX_dict_clean['Open']

In [109]:
max_open = max(AFX_dict_clean['Open'])
max_open

53.11

In [110]:
#find minimum positive open price since None values were replaced with -1
min_open = min(i for i in AFX_dict_clean['Open'] if i > 0)
min_open

34.0

In [111]:
#calculate missing Change values
for index, cp in enumerate(AFX_dict_clean['Change']):
    AFX_dict_clean['Change'][index] = (AFX_dict_clean['High'][index]-AFX_dict_clean['Low'][index])

In [112]:
#maximum in-a-day change based on High and Low price for the day
max_day_change = max(AFX_dict_clean['Change'])
max_day_change

2.8100000000000023

In [113]:
#date of max in-a-day change
AFX_dict_clean['Date'][AFX_dict_clean['Change'].index(max_day_change)]


'2017-05-11'

In [114]:
#calculate new list with change in price between adjacent days
AFX_dict_clean['Inter-Day Change'] = []
for i in range(1, len(AFX_dict_clean['Close'])):
    AFX_dict_clean['Inter-Day Change'].append(AFX_dict_clean['Close'][i]-AFX_dict_clean['Close'][i-1])

In [115]:
#absolute maximum change in price between two adjacent days
max_interday_change = max(AFX_dict_clean['Inter-Day Change'], key = abs)
max_interday_change

2.559999999999995

In [116]:
#date of max interday change in price
AFX_dict_clean['Date'][AFX_dict_clean['Inter-Day Change'].index(max_interday_change)]

'2017-08-09'

In [117]:
#average daily trade volume
avg_daily_vol = sum(AFX_dict_clean['Traded Volume'])/len(AFX_dict_clean['Traded Volume'])
avg_daily_vol

89124.33725490196

In [118]:
#median trading volume
def median(lst):
    lst.sort()
    med1 = lst[len(lst)//2]
    med2 = lst[len(lst)//2 - 1]
    med = (med1 + med2)/2
    return med

median(AFX_dict_clean['Traded Volume'])
    

75630.5